# BingSearch using Kernel 

This notebook explains how to integrate Bing Search with the Semantic Kernel  to get the latest information from the internet.

To use Bing Search you simply need a Bing Search API key. You can get the API key by creating a [Bing Search resource](https://learn.microsoft.com/en-us/bing/search-apis/bing-web-search/create-bing-search-service-resource) in Azure. 

To learn more read the following [documentation](https://learn.microsoft.com/en-us/bing/search-apis/bing-web-search/overview)


Prepare a Semantic Kernel instance first, loading also the AI backend settings defined in the [Setup notebook](0-AI-settings.ipynb):

In [29]:
#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.0.0-beta"

#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Plugins.Core;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.Planning;
using Microsoft.SemanticKernel.Planners.Sequential;
using Microsoft.SemanticKernel.TemplateEngine;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;
var builder = new KernelBuilder();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.WithAzureChatCompletionService(model, azureEndpoint, apiKey);
else
    builder.WithOpenAIChatCompletionService(model, apiKey, orgId);

var kernel = builder.Build();

Error: Microsoft.SemanticKernel version 1.0.0-beta cannot be added because version 1.0.0-beta1 was added previously.

Add the following namespaces to use the Bing Search connector.

In [2]:
using Microsoft.SemanticKernel.Plugins.Web;
using Microsoft.SemanticKernel.Plugins.Web.Bing;

Enter your Bing Search Key in BING_KEY using `InteractiveKernel` method as introduced in [`.NET Interactive`](https://github.com/dotnet/interactive/blob/main/docs/kernels-overview.md)

In [43]:
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

var BING_KEY = await InteractiveKernel.GetPasswordAsync("Please enter your Bing Search Key");

Below are some examples on how [`WebSearchEngineSkill`](../src/Skills/Skills.Web/WebSearchEngineSkill.cs) can be used in SK. 

In [31]:
private static async Task Example1Async(IKernel kernel)
{
        Console.WriteLine("Example 1");

        // Run 
        var question = "What is quantum tunnelling";
        var bingResult = await kernel.Functions.GetFunction("bing", "search").InvokeAsync(kernel.CreateNewContext(new ContextVariables(question)));

        Console.WriteLine(question);
        Console.WriteLine("----");
        Console.WriteLine(bingResult.GetValue<string>());
        Console.WriteLine();

        /* OUTPUT:

            What is quantum tunnelling
            ----
            In physics, quantum tunnelling, barrier penetration, or simply tunnelling is a quantum mechanical
            phenomenon in which an object such as an electron or atom passes through a potential energy 
            barrier that, according to classical mechanics, the object does not have sufficient energy to
            enter or surmount.
            
       */
}

In [32]:
private static async Task Example2Async(IKernel kernel)
{
    Console.WriteLine("Example 2");

    //The following function only works in interactive notebooks
    string question = await InteractiveKernel.GetInputAsync("Please ask your question"); 

    var bingResult = await kernel.Functions.GetFunction("bing", "search").InvokeAsync(kernel.CreateNewContext(new ContextVariables(question)));

    Console.WriteLine(bingResult.GetValue<string>());
}

Just uncomment the examples to run them in the following code:

In [46]:
// Load Bing skill
var bingConnector = new BingConnector(BING_KEY.GetClearTextPassword());

kernel.ImportFunctions(new WebSearchEnginePlugin(bingConnector), "bing");

await Example1Async(kernel);
//await Example2Async(kernel);

Example 1
What is quantum tunnelling
----
["In physics, quantum tunnelling, barrier penetration, or simply tunnelling is a quantum mechanical phenomenon in which an object such as an electron or atom passes through a potential energy barrier that, according to classical mechanics, the object does not have sufficient energy to enter or surmount.","Quantum tunnelling is a phenomenon where a subatomic particle or atom can be present on the opposite side of the physical obstacle that is normally impossible for the particle to infiltrate. This is as if a person is walking and reaching a 20-metre tall wall extending a long way on both sides.","Quantum tunneling is a phenomenon where an electron is able to phase through a barrier and move to the other side. It is a quantum phenomenon that occurs when particles move through a barrier that, according to the theories of classical physics, should be impossible to pass through.","One of the strangest things scientists discovered when picking apart

In [45]:
BING_KEY

************